In [ ]:
import transformers
from transformers import AutoModel, AutoImageProcessor
from transformers import pipeline
from transformers import SegformerImageProcessor, SegformerForSemanticSegmentation, BaseImageProcessor
import numpy
import pathlib as pl
import PIL 
import datasets
from datasets import Dataset
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
image_processor = SegformerImageProcessor.from_json_file("./sip/preprocessor_config.json")
model = SegformerForSemanticSegmentation.from_pretrained("glacierscopessegmentation/glacier_segmentation_transformer")
data_path = pl.Path("./inference_dataset/inference_dataset.hf/")
segmenter = pipeline(task="image-segmentation",image_processor=image_processor,model = model,device = "cuda:0")

In [ ]:
from datasets import disable_caching
import datasets
import os
os.system("export HF_DATASETS_CACHE=/home/groups/dustinms/aashrayc/.cache")
os.system('export HF_HOME="/home/groups/dustinms/aashrayc/.cache"')
os.system('export HF_DATASETS_CACHE="/home/groups/dustinms/aashrayc/.cache"')
os.environ["HF_DATASETS_CACHE"] = "/home/groups/dustinms/aashrayc/.cache"
os.environ["HF_HOME"] = "/home/groups/dustinms/aashrayc/.cache"
os.environ["HF_DATASETS_CACHE"] = "/home/groups/dustinms/aashrayc/.cache"

disable_caching()
ds = datasets.load_dataset("glacierscopessegmentation/scopes_test",download_config=datasets.DownloadConfig(delete_extracted=True,extract_on_the_fly=True, cache_dir = "/home/groups/dustinms/aashrayc/.cache/"), cache_dir="/home/groups/dustinms/aashrayc/.cache/")  
newds = datasets.load_from_disk(data_path)


In [ ]:
for c, i in enumerate(ds["train"]):
    segments = segmenter(i["image"])
    red = np.array(segments[0]["mask"].convert("L"))
    green = np.array(segments[2]["mask"].convert("L"))
    blue = np.array(segments[1]["mask"].convert("L"))
    combo = np.stack([red, green, blue]).transpose(1,2,0)/255

    f, (ax1, ax2, ax3) = plt.subplots(1, 3, sharey=True, figsize = (18,6))

    img1 = ax1.imshow(combo)
    img2 = ax2.imshow(i["image"])
    img3 = ax3.imshow(np.array(i["label"])*80)
    plt.colorbar(img1)
    plt.colorbar(img2)
    plt.colorbar(img3)
    plt.show()
    if c > 10: break

In [ ]:
for c, i in enumerate(newds):
    segments = segmenter(i["image"])
    red = np.array(segments[0]["mask"].convert("L"))
    green = np.array(segments[2]["mask"].convert("L"))
    blue = np.array(segments[1]["mask"].convert("L"))
    combo = np.stack([red, green, blue]).transpose(1,2,0)/255
    base = np.array(i["image"])
    f, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize = (18,9))
    img1 = ax1.imshow(combo)
    img2 = ax2.imshow(base)
    plt.colorbar(img1)
    plt.colorbar(img2)
    plt.show()
    print(c)
